In [1]:
from pyspark.mllib.clustering import KMeans
from numpy import array
from math import sqrt
from pyspark.sql import SQLContext
import json
import sys
import os
import calendar
import time
import base64
import requests
import math

#Globals for Kafka config
OUTPUTTOPIC='newOutput'
containerIP='134.168.1.164'

URL = 'http://' + containerIP + '9443/topics/' + OUTPUTTOPIC
HEADERS = {'Content-Type': 'application/vnd.kafka.v1+json'}

# x mins ago of the current time
min_offset = 240

#Epoch time 5 mins ago as of the current time
newEpochOffset = (calendar.timegm(time.gmtime()) - min_offset * 60) * 1000

#last N entries in the database
limitN = 20

#Sensors to analyze
sensors = ['PM41301', 'PM41801', 'PM107631']
def do_the_math(SensorID):
    df1 = sqlContext.sql("SELECT codigo, intensidad, velocidad, tf, dt FROM madridtraffic WHERE codigo = \'" + SensorID + "\'")
    df1.show()
    df1 = df1[df1['codigo'] == SensorID]
    print df1.show()
    if df1.rdd.isEmpty():
        print "There's no data set for the specified sensors."
        return 
    #accessing data of one sensor
    
    #mllib clustering function requires data in array and float valuesdf1
    parsedData = df1.map(lambda row: array([float(row.intensidad), float(row.velocidad)]))
    print('parsedData is mapped')
    
    #calculate mean for standardization
    mean = parsedData.mean()
    #print mean
    #calculate standard deviation
    std = parsedData.stdev()

    processedData = parsedData.map(lambda x: ((x-mean)/std))
    print 'normalized the data'

    ## Aren't we blowing up this data each iteration?  Do we really care?
    # Build the model (cluster the data)
    clusters = KMeans.train(processedData, 2, maxIterations=10, runs=1, initializationMode="random")
    print 'clusters initilized'


    ##What is this really doing?
    #denormalize the centers.
    centers = (clusters.centers*std)+mean
    print 'denormalized the centers'


    #midpoints-threshold
    a = (centers[0][0]+centers[1][0])/2
    b = (centers[0][1]+centers[1][1])/2

    #creates dictionary
    message = {"codigo": SensorID, "newThresholds":{"ThresholdSpeed":int(b), "ThresholdIntensity":int(a)}}
    print message
    message = base64.b64encode(str(message))

    #creates dictionary
    messages = {"records":[{"value":message}]}
    #converts dictionary into json msg
    Threshold = json.dumps(messages)
    
    #publish the message: payload is threshold and other field is topic
    r = requests.post(URL, data=Threshold, headers=HEADERS)
    print(r.text)
    
def kick_off_routine():
    #Loop through the sensors
    for i in sensors:
        do_the_math(i)

while True:
    df = sqlContext.read.parquet("swift://DataServices.spark/DataServices")
    df.show()

    df = df.cache()
    df.registerTempTable("madridtraffic")

    kick_off_routine()
    df.unpersist()
    time.sleep(300)

Py4JJavaError: An error occurred while calling o30.parquet.
: java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.metastore.HiveMetaStoreClient
	at org.apache.hadoop.hive.ql.session.SessionState.start(SessionState.java:346)
	at org.apache.spark.sql.hive.client.ClientWrapper.<init>(ClientWrapper.scala:116)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:86)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:58)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:542)
	at org.apache.spark.sql.hive.client.IsolatedClientLoader.liftedTree1$1(IsolatedClientLoader.scala:172)
	at org.apache.spark.sql.hive.client.IsolatedClientLoader.<init>(IsolatedClientLoader.scala:168)
	at org.apache.spark.sql.hive.HiveContext.metadataHive$lzycompute(HiveContext.scala:213)
	at org.ap